In [1]:
import time
import random
import sys
import os
import glob
import logging
import numpy as np
import cv2

In [2]:
CARLA_PATH = 'C:/Users/saile/Desktop/Sailesh/Carla Simulator/CARLA_0.9.10/WindowsNoEditor/PythonAPI/carla/dist/carla-*%d.%d-%s.egg'
try:
    sys.path.append(glob.glob(CARLA_PATH % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla
from carla import VehicleLightState as vls 

In [3]:
SHOW_PREVIEW = False
VEHICLE_NAME = 'audi'
MODEL_NAME = 'a2'

In [4]:
actor_list = []

In [5]:
client = carla.Client('localhost', 2000)
world = client.get_world()
map = world.get_map()
blueprint_library = world.get_blueprint_library()
spawn_points = map.get_spawn_points()

In [6]:
vehicle_blueprint = blueprint_library.filter(f"vehicle.{VEHICLE_NAME}.{MODEL_NAME}")[0]
vehicle_spawn_location = random.choice(map.get_spawn_points())
vehicle = world.try_spawn_actor(vehicle_blueprint, vehicle_spawn_location)
actor_list.append(vehicle)

In [7]:
rgb_camera_bp = blueprint_library.find('sensor.camera.rgb')
rgb_camera_bp.set_attribute("image_size_x", "600")
rgb_camera_bp.set_attribute("image_size_y", "400")
rgb_camera_bp.set_attribute("fov", "110")
depth_sensor_bp = blueprint_library.find('sensor.camera.depth')
depth_sensor_bp.set_attribute("image_size_x", "600")
depth_sensor_bp.set_attribute("image_size_y", "400")
depth_sensor_bp.set_attribute("fov", "120")
rgb_camera_transform = carla.Transform(carla.Location(x=3, z=2))
depth_sensor_transform = carla.Transform(carla.Location(x=3, z=2))
rgb_camera = world.spawn_actor(rgb_camera_bp, rgb_camera_transform, attach_to=vehicle)
depth_sensor = world.spawn_actor(depth_sensor_bp, depth_sensor_transform, attach_to=vehicle)

In [8]:
def show_image(image):
    img = np.array(image.raw_data)
    img = img.reshape((400, 600, 4))
    img = img[:,:,:3]
    cv2.imshow("car image", img)
    cv2.waitKey(1)
depth_sensor.listen(lambda image: show_image(image))